In [51]:
# STEP 1: Install dependencies
!apt-get install -y poppler-utils tesseract-ocr
!pip install pdf2image pytesseract opencv-python-headless matplotlib ezdxf rapidfuzz


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [52]:
# STEP 2: Import libraries
import os
import json
import cv2
import pytesseract
import numpy as np
import ezdxf
from rapidfuzz import process
from pdf2image import convert_from_path
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [53]:
# STEP 3: Upload your PDF
from google.colab import files
uploaded = files.upload()

pdf_file = list(uploaded.keys())[0]

Saving Pages from WA13_90__IH69 CTMS.pdf to Pages from WA13_90__IH69 CTMS (5).pdf


In [54]:
# STEP 4: Convert first N pages of PDF to images
images = convert_from_path(pdf_file, dpi=300, first_page=1, last_page=3)
os.makedirs("legend_images", exist_ok=True)

image_paths = []
for i, img in enumerate(images):
    path = f"legend_images/page_{i+1}.png"
    img.save(path, "PNG")
    image_paths.append(path)

In [61]:
# STEP 5: Manually select legend area from first page using matplotlib
gate = input("
⚠️  Press ENTER once you're ready to select the legend area...")
selected_coords = []
def onclick(event):
    selected_coords.append((int(event.xdata), int(event.ydata)))
    if len(selected_coords) == 2:
        plt.close()

legend_crops = []
print("Please click two corners (top-left and bottom-right) of the legend area")
img = cv2.imread(image_paths[0])
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(rgb_img)
plt.title("Click top-left and bottom-right of legend")
plt.connect('button_press_event', onclick)
plt.show()

if len(selected_coords) == 2:
    (x1, y1), (x2, y2) = selected_coords
    x, y, w, h = x1, y1, x2 - x1, y2 - y1
    crop = img[y:y+h, x:x+w]
    legend_crops.append(crop)
    cv2.imwrite("legend_images/legend_manual.png", crop)

SyntaxError: unterminated string literal (detected at line 2) (<ipython-input-61-ca996ed63a7d>, line 2)

In [56]:
# STEP 6: Split legend into rows (increased height)
def split_legend_rows(img, row_height=65):
    rows = []
    h = img.shape[0]
    for i in range(0, h, row_height):
        row = img[i:i+row_height, :]
        rows.append(row)
    return rows



In [57]:
# STEP 7: Extract text and symbol, apply fuzzy matching with timeout handling, save to JSON
output = []
os.makedirs("legend_items", exist_ok=True)

known_legend_items = [
    "PROPOSED FIBER CABINET",
    "EXISTING FIBER CABINET",
    "PROPOSED CCTV CAMERA",
    "EXISTING CCTV CAMERA",
    "PROPOSED CONDUIT",
    "PROPOSED CONDUIT (BORE)",
    "EXISTING MANHOLE",
    "PROPOSED MANHOLE",
    "EXISTING CONDUIT",
    "EXISTING CONDUIT (BORE)",
    "WRONG WAY DRIVER DETECTION"
]

for i, crop in enumerate(legend_crops):
    rows = split_legend_rows(crop)
    for j, row in enumerate(rows):
        symbol = row[:, :60]  # Left section
        label_img = row[:, 60:]  # Right section

        symbol_path = f"legend_items/symbol_{i}_{j}.png"
        cv2.imwrite(symbol_path, symbol)

        gray = cv2.cvtColor(label_img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        _, binary = cv2.threshold(resized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        try:
            text = pytesseract.image_to_string(binary, config='--psm 7').strip()
        except Exception as e:
            print(f"[!] OCR error on row {i}-{j}: {e}")
            continue

        if text:
            best_match, score, _ = process.extractOne(text.upper(), known_legend_items)
            if score > 70:
                final_label = best_match
            else:
                final_label = f"UNMATCHED: {text.upper()}"

            print(f"OCR: {repr(text)} → Match: {repr(final_label)} (Score: {score})")

            output.append({
                "level_name": final_label,
                "symbol_image": symbol_path
            })


In [58]:
# STEP 8: Save JSON file
with open("legend_levels.json", "w") as f:
    json.dump(output, f, indent=2)

files.download("legend_levels.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
# STEP 9: Create DWG of legend items
os.makedirs("dwg_output", exist_ok=True)
doc = ezdxf.new()
msp = doc.modelspace()

# Title text
text_entity = msp.add_text("Legend Items", dxfattribs={"height": 2.5})
text_entity.dxf.insert = (0, 0)

# Add each legend row
y = -5
for item in output:
    level_name = item["level_name"]
    msp.add_line((0, y), (4, y), dxfattribs={"color": 1})  # Placeholder line symbol
    label = msp.add_text(level_name, dxfattribs={"height": 1.5})
    label.dxf.insert = (5, y - 0.5)
    y -= 3

# Save DWG
dwg_path = "dwg_output/legend_items.dwg"
doc.saveas(dwg_path)
print(f"\n✅ DWG legend sheet saved as '{dwg_path}'")
files.download(dwg_path)



✅ DWG legend sheet saved as 'dwg_output/legend_items.dwg'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>